In [1]:
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

import numpy as np
import pandas as pd

# Data import

In [4]:
with open("text-data.json", "r") as filehandle:
    data = json.load(filehandle)

# Data Parsing

In [5]:
data_parsed = [entry.replace(", ", ",") for entry in data]
data_parsed = [entry.split(",") for entry in data_parsed]
data_parsed = pd.DataFrame.from_dict(data_parsed)
data_parsed.columns = ["timestamp", "x", "temp", "humid"]
# Convert temp and humid to numeric
data_parsed["temp"] = data_parsed["temp"].astype(float)
data_parsed["humid"] = data_parsed["humid"].astype(float)
# Last row is empty
data_parsed = data_parsed[:-1]
# Remove X
data_parsed.drop(columns="x", inplace=True)
# Convert to datetime
data_parsed["timestamp"] = pd.to_datetime(data_parsed["timestamp"])


# Data Stats

In [ ]:
# Time range
print(data_parsed["timestamp"].min(), data_parsed["timestamp"].max())
# Mean Temp per weekday
print(data_parsed.groupby(
    [data_parsed["timestamp"].dt.weekday])["temp"].mean())
# Mean Humid per weekday
print(data_parsed.groupby(
    [data_parsed["timestamp"].dt.weekday])["humid"].mean())

# Feature Enhancement

In [497]:
# Add hour
data_parsed["hour"] = data_parsed["timestamp"].dt.hour
# Add day of year
data_parsed["day_of_year"] = data_parsed["timestamp"].dt.day_of_year
# Add weekday
data_parsed["weekday"] = data_parsed["timestamp"].dt.weekday

# ML

In [ ]:
class StartNet(nn.Module):

    def __init__(self):
        super(StartNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 15),
            nn.ReLU(),
            nn.Linear(15, 1))

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
class MyDataset(torch.utils.data.Dataset):

  def __init__(self, df):
 
    x = df[["hour", "weekday", "day_of_year"]].values
    y = df[["temp"]].values

    self.x_train=torch.tensor(x, dtype=torch.float32)
    self.y_train=torch.tensor(y, dtype=torch.float32)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self, idx):
    return self.x_train[idx], self.y_train[idx]

In [498]:
def training_loop(n_epochs, optimiser, model, loss_fn, train_dl, val_dl):
    for epoch in range(1, n_epochs + 1):
        model.train()
        for i, data in enumerate(train_dl):
            optimiser.zero_grad() # set gradients to zero
            inputs, targets = data
            output_train = model(inputs) # forwards pass
            loss_train = loss_fn(output_train, targets) # calculate loss
            loss_train.backward() # backwards pass
            optimiser.step() # update model parameters

        model.eval()
        for i, data in enumerate(val_dl):
            inputs, targets = data
            output_val = model(inputs)
            loss_val = loss_fn(output_val, targets)
        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                f" Validation loss {loss_val.item():.4f}")

In [499]:
startnet = StartNet()
optimizer = optim.Adam(startnet.parameters(), lr=0.001)

In [500]:
# Train Val Split
train = data_parsed[0:int(len(data_parsed)/1.5)]
val = data_parsed[int(len(data_parsed)/1.5):]

In [501]:
train_dl  = torch.utils.data.DataLoader(MyDataset(train), batch_size=10, shuffle=False)
val_dl    = torch.utils.data.DataLoader(MyDataset(val), batch_size=10, shuffle=False)

In [502]:
training_loop(
    n_epochs = 700, 
    optimiser = optimizer,
    model = startnet,
    loss_fn = nn.MSELoss(),
    train_dl = train_dl,
    val_dl = val_dl,
    )

Epoch 1, Training loss 2.5529, Validation loss 0.2576
Epoch 100, Training loss 0.7277, Validation loss 2.7463
Epoch 200, Training loss 0.7324, Validation loss 3.1789
Epoch 300, Training loss 0.7045, Validation loss 3.2853
Epoch 400, Training loss 0.6785, Validation loss 3.2429
Epoch 500, Training loss 0.6549, Validation loss 3.1821
Epoch 600, Training loss 0.6320, Validation loss 3.1198
Epoch 700, Training loss 0.6111, Validation loss 3.0603


In [506]:
startnet.eval()
startnet(torch.tensor([15.0, 6, 300]))

tensor([16.5288], grad_fn=<AddBackward0>)

In [508]:
torch.save(startnet, "startnet.model") 